In [1]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@6818214b


Some(http://iuni2:4040)

In [2]:
val WoS = spark.read.format("parquet").load("/WoSraw/parquet/*.parquet")

val WoS2 = WoS.select("UID","static_data.summary.titles","static_data.fullrecord_metadata.category_info")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- category_info: struct (nullable = true)
 |    |-- headings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- heading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subheadings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subheading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subjects: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subject: array (nullable = true)
 |    |    |    |-- element: struct (con

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 1 more field]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 1 more field]

In [4]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)
val sdf = spark.read.format("csv")
      .option("header", "true")
      .option("quote", "\"")
      .option("escape", "\"")
      //.option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/metadata.csv")
sdf.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+
|  count|  137411|63458|  137411|137370|113040|70461|   100662| 137411|  105896|      137397| 132101| 129973|     0|           26163|    1686|         63458|         50801|125347|112844|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+



session = org.apache.spark.sql.SparkSession@fda30ef
azureFile = /azureBlob.txt MapPartitionsRDD[22] at textFile at <console>:58
sdf = [cord_uid: string, sha: string ... 17 more fields]


[cord_uid: string, sha: string ... 17 more fields]

In [3]:
//sdf.printSchema
sdf.filter("sha is not null").show(false)

+--------+----------------------------------------+--------+--------------------------------------------------------------------------------------------------------------------------------+------------------------+---------+---------+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
val magPapers = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/mag/Papers.txt")
magPapers.show

+-----+-----+--------------------+------+-----------------------------------+-----------------------------------+----+----+----------+----------------------------+----+----+----+----+----+----+----+----+----+----+----------------------------------+----+-------------------+
|  _c0|  _c1|                 _c2|   _c3|                                _c4|                                _c5| _c6| _c7|       _c8|                         _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|                              _c20|_c21|               _c22|
+-----+-----+--------------------+------+-----------------------------------+-----------------------------------+----+----+----------+----------------------------+----+----+----+----+----+----+----+----+----+----+----------------------------------+----+-------------------+
| 8933|30118|                null|  null|               историческая наук...|               Историческая наук...|null|1960|1960-10-31|        ТОО Редакция журн...|null|null|null|

magPapers = [_c0: bigint, _c1: int ... 21 more fields]


lastException: Throwable = null


[_c0: bigint, _c1: int ... 21 more fields]

In [41]:
val doiMatch = sdf.join(magPapers, sdf("doi")===magPapers("_c2"), "left").cache()
doiMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|  url| s2_id|  _c0|  _c1|  _c2|  _c3|  _c4|  _c5|_c6|  _c7|  _c8|  _c9| _c10|_c11|_c12|_c13|_c14| _c15| _c16| _c17| _c18| _c19| _c20|_c21|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|  count|  128648|55847|  128648|128620|100773|60927|    99169| 128

doiMatch = [cord_uid: string, sha: string ... 40 more fields]


[cord_uid: string, sha: string ... 40 more fields]

In [6]:
val idMap = spark.read.format("csv").option("header", false).load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/2020-06-08-CORD-UID-MappedTo-2020-05-29-MAG-ID.csv")
val idMatch = sdf.join(idMap, sdf("cord_uid")===idMap("_c0"), "left").withColumnRenamed("_c1","MAGid").drop("_c0")
idMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id| MAGid|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+
|  count|  139157|64190|  139157|139114|114769|71292|   101858| 139157|  107108|      139142| 133817| 131709|     0|           27366|    1691|         64190|         51366|127093|114564|120866|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+



idMap = [_c0: string, _c1: string]
idMatch = [cord_uid: string, sha: string ... 18 more fields]


lastException: Throwable = null


[cord_uid: string, sha: string ... 18 more fields]

In [20]:
val idMatch2 = idMatch.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid")) as "MAGids")
idMatch2.describe().filter($"summary" === "count").show

+-------+--------+------+
|summary|cord_uid|MAGids|
+-------+--------+------+
|  count|  136783|136783|
+-------+--------+------+



idMatch2 = [cord_uid: string, MAGids: string]


[cord_uid: string, MAGids: string]

In [70]:
val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
magMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+------+------+------+------+------+------+------+---+------+------+------+------+----+----+-----+-----+-----+-----+------+------+------+------+----+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|  url| s2_id| MAGid|   _c0|   _c1|   _c2|   _c3|   _c4|   _c5|_c6|   _c7|   _c8|   _c9|  _c10|_c11|_c12| _c13| _c14| _c15| _c16|  _c17|  _c18|  _c19|  _c20|_c21|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+------+------+------+------+------+------+------+---+------+------+------+------+----+----+-----+-----+-----+-----+------+------+------+------+----

magMatch = [cord_uid: string, sha: string ... 41 more fields]


[cord_uid: string, sha: string ... 41 more fields]

In [7]:
val magPPA = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/mag/PaperAuthorAffiliations.txt")

magPPA.show

+---+----------+----------+---+--------------------+--------------------+
|_c0|       _c1|       _c2|_c3|                 _c4|                 _c5|
+---+----------+----------+---+--------------------+--------------------+
|  9|2632942543|      null|  1|Victoriano Perruc...|                null|
| 15| 199142497|      null|  1|     Robert Münscher|Heidelberg, Deuts...|
| 15| 680395887|      null|  2|       Julia Hormuth|Reutlingen, Deuts...|
| 23|1243978490|  79576946|  1|     Eric T Stoopler|Department of Ora...|
| 23|2582258949|      null|  3| Arthur S Kuperstein|                null|
| 23|2582736345|      null|  2|        Ying Wai Sia|                null|
| 58|2641488431|      null|  1|             I. Uray|                null|
| 58|2693978806|      null|  2|           P. Kovacs|                null|
| 79|2662843304|      null|  1|           K. Suresh|                null|
| 79|2683440697|      null|  2|     T. S. J. Norman|                null|
| 87|2676159639|      null|  1| Gabrie

magPPA = [_c0: bigint, _c1: bigint ... 4 more fields]


[_c0: bigint, _c1: bigint ... 4 more fields]

In [8]:
val magPPA2 = magPPA.toDF("MAGid2","authorID","affiliationID","authorOrder","author","affiliation").filter("affiliationID is not null")
val magMatchPPA = idMatch.join(magPPA2, idMatch("MAGid")===magPPA2("MAGid2"), "left").cache()
magMatchPPA.describe().filter($"summary" === "count").show

+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+------+--------+-------------+-----------+------+-----------+
|summary|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|
+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+------+--------+-------------+-----------+------+-----------+
|  count|  538235|288196|  538235|538175|485223|288319|   438353| 538235|  462375|      538188| 526410| 510583|     0|           79027|    2662|        288196|        228538|509964|455950|519944|496131|  496131|       496131|    

magPPA2 = [MAGid2: bigint, authorID: bigint ... 4 more fields]
magMatchPPA = [cord_uid: string, sha: string ... 24 more fields]


[cord_uid: string, sha: string ... 24 more fields]

In [9]:
val magAffiliation = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/mag/Affiliations.txt")

magAffiliation.show

+--------+-----+--------------------+--------------------+-------------+--------------------+--------------------+-----+-----+-------+----------+----------+-------------------+
|     _c0|  _c1|                 _c2|                 _c3|          _c4|                 _c5|                 _c6|  _c7|  _c8|    _c9|      _c10|      _c11|               _c12|
+--------+-----+--------------------+--------------------+-------------+--------------------+--------------------+-----+-----+-------+----------+----------+-------------------+
|20455151| 9927|         air liquide|         Air Liquide|grid.476009.c|https://web.archi...|http://en.wikiped...| 6631| 6624|  54522|52.5035934|  -1.80516|2016-06-24 00:00:00|
|24386293|13883|hellenic national...|Hellenic National...|         null|http://www.hnms.g...|http://en.wikiped...|   84|   84|   1830|  37.97614|   23.7364|2016-06-24 00:00:00|
|32956416|12955|catholic universi...|Catholic Universi...|grid.448708.7|  http://www.uco.fr/|http://en.wikiped...| 

magAffiliation = [_c0: bigint, _c1: int ... 11 more fields]


[_c0: bigint, _c1: int ... 11 more fields]

In [10]:
val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c9","_c10","_c11").toDF("affiliationID2","normalizedName","displayName","GRIDid","wiki","paperTotal", "citationTotal", "Latitude", "Longitude")
val magMatchPPA2 = magMatchPPA.join(magAffiliation2, magMatchPPA("affiliationID")===magAffiliation2("affiliationID2"), "left").cache()
magMatchPPA2.describe().filter($"summary" === "count").show

+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+------+--------+-------------+-----------+------+-----------+--------------+--------------+-----------+------+------+----------+-------------+--------+---------+
|summary|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|affiliationID2|normalizedName|displayName|GRIDid|  wiki|paperTotal|citationTotal|Latitude|Longitude|
+-------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+------+------+--------+-------------+-----------+------+-----------+--------------+--------------+

magAffiliation2 = [affiliationID2: bigint, normalizedName: string ... 7 more fields]
magMatchPPA2 = [cord_uid: string, sha: string ... 33 more fields]


[cord_uid: string, sha: string ... 33 more fields]

In [11]:
val PaperMAG = magMatchPPA2.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    concat_ws(";", collect_list("authorID")) as "authorids",
                                                    concat_ws(";", collect_list("authorOrder")) as "authorOrders",
                                                    concat_ws(";", collect_list("affiliationID2")) as "affiliationids",
                                                    concat_ws(";", collect_list("normalizedName")) as "affiliationNormalized",
                                                    concat_ws(";", collect_list("displayName")) as "affiliationNames",
                                                    concat_ws(";", collect_list("GRIDid")) as "GRIDids")
PaperMAG.show(false)

+--------+-----------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

PaperMAG = [cord_uid: string, MAGids: string ... 6 more fields]


[cord_uid: string, MAGids: string ... 6 more fields]

In [19]:
PaperMAG.describe().filter($"summary" === "count").show

+-------+--------+------+---------+------------+--------------+---------------------+----------------+-------+
|summary|cord_uid|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|
+-------+--------+------+---------+------------+--------------+---------------------+----------------+-------+
|  count|  136783|136783|   136783|      136783|        136783|               136783|          136783| 136783|
+-------+--------+------+---------+------------+--------------+---------------------+----------------+-------+



In [22]:
PaperMAG.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/CORD-19-MAGinsts.csv")

In [14]:
val insititutionMAG = magMatchPPA2.groupBy("affiliationID").agg(//concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    //concat_ws(";", collect_list("cord_uid")) as "cord_uids",
                                                    first("displayName") as "displayName",
                                                    first("normalizedName") as "normalizedName",
                                                    first("wiki") as "wiki",
                                                    first("paperTotal") as "paperTotal", 
                                                    first("citationTotal") as "citationTotal",
                                                    first("Latitude") as "Latitude",
                                                    first("Longitude") as "Longitude",
                                                    first("GRIDid") as "GRIDid")
insititutionMAG.show(false)

+-------------+------------------------------------------+------------------------------------------+-----------------------------------------------------------------------+----------+-------------+-----------+-----------+-------------+
|affiliationID|displayName                               |normalizedName                            |wiki                                                                   |paperTotal|citationTotal|Latitude   |Longitude  |GRIDid       |
+-------------+------------------------------------------+------------------------------------------+-----------------------------------------------------------------------+----------+-------------+-----------+-----------+-------------+
|746986       |Halmstad University                       |halmstad university                       |http://en.wikipedia.org/wiki/Halmstad_University,_Sweden               |2481      |37490        |56.66417   |12.87861   |grid.73638.39|
|6141010      |European Institute of Oncology       

insititutionMAG = [affiliationID: bigint, displayName: string ... 7 more fields]


[affiliationID: bigint, displayName: string ... 7 more fields]

In [23]:
insititutionMAG//.select("wosid","headings","subheadings","WC","SC","disciplines","subdisciplines","fractions")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/MAGInstTable.csv")

In [21]:
val pdfFull = spark.read.format("json")
      //.option("inferSchema","true")
      .load("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/pdf_json/*.json")
pdfFull.printSchema

root
 |-- _corrupt_record: string (nullable = true)
 |-- abstract: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cite_spans: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- ref_id: string (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- ref_spans: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- ref_id: string (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |-- section: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- back_matter: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 

pdfFull = [_corrupt_record: string, abstract: array<struct<cite_spans:array<struct<end:bigint,ref_id:string,start:bigint,text:string>>,ref_spans:array<struct<end:bigint,ref_id:string,start:bigint,text:string>>,section:string,text:string>> ... 6 more fields]


[_corrupt_record: string, abstract: array<struct<cite_spans:array<struct<end:bigint,ref_id:string,start:bigint,text:string>>,ref_spans:array<struct<end:bigint,ref_id:string,start:bigint,text:string>>,section:string,text:string>> ... 6 more fields]

In [24]:
val pdfAuthors = pdfFull.select("paper_id", "metadata.title", "metadata.authors").cache()
pdfAuthors.describe().filter($"summary" === "count").show

+-------+--------+-----+
|summary|paper_id|title|
+-------+--------+-----+
|  count|   26201|26201|
+-------+--------+-----+



pdfAuthors = [paper_id: string, title: string ... 1 more field]


[paper_id: string, title: string ... 1 more field]

In [95]:
pdfAuthors.printSchema

root
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- affiliation: struct (nullable = true)
 |    |    |    |-- institution: string (nullable = true)
 |    |    |    |-- laboratory: string (nullable = true)
 |    |    |    |-- location: struct (nullable = true)
 |    |    |    |    |-- addrLine: string (nullable = true)
 |    |    |    |    |-- country: string (nullable = true)
 |    |    |    |    |-- postBox: string (nullable = true)
 |    |    |    |    |-- postCode: string (nullable = true)
 |    |    |    |    |-- region: string (nullable = true)
 |    |    |    |    |-- settlement: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- first: string (nullable = true)
 |    |    |-- last: string (nullable = true)
 |    |    |-- middle: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    

In [98]:
pdfAuthors.filter("authors.affiliation is not null").select("authors.affiliation.institution", "authors.affiliation.laboratory").show(false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|institution                                                                                                                                                                                                                                                                                                                                                                             

In [68]:
val url = "https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb"
val result = scala.io.Source.fromURL(url).mkString
println(result)

{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) w

url = https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb
result = 


"{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred yea...
